In [2]:
import tempfile
import os

import tensorflow as tf
import numpy as np

from tensorflow import keras

%load_ext tensorboard

In [3]:
# Load MNIST dataset
mnist = keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Normalize the input image so that each pixel value is between 0 and 1.
train_images = train_images / 255.0
test_images = test_images / 255.0

# Define the model architecture.
model = keras.Sequential([
  keras.layers.InputLayer(input_shape=(28, 28)),
  keras.layers.Reshape(target_shape=(28, 28, 1)),
  keras.layers.Conv2D(filters=12, kernel_size=(3, 3), activation='relu'),
  keras.layers.MaxPooling2D(pool_size=(2, 2)),
  keras.layers.Flatten(),
  keras.layers.Dense(10)
])

# Train the digit classification model
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.fit(
  train_images,
  train_labels,
  epochs=4,
  validation_split=0.1,
)

Epoch 1/4
1688/1688 [==============================] - 6s 4ms/step - loss: 0.2850 - accuracy: 0.9196 - val_loss: 0.1139 - val_accuracy: 0.9692
Epoch 2/4
1688/1688 [==============================] - 6s 4ms/step - loss: 0.1163 - accuracy: 0.9664 - val_loss: 0.0819 - val_accuracy: 0.9777
Epoch 3/4
1688/1688 [==============================] - 6s 4ms/step - loss: 0.0884 - accuracy: 0.9741 - val_loss: 0.0786 - val_accuracy: 0.9778
Epoch 4/4
1688/1688 [==============================] - 6s 4ms/step - loss: 0.0724 - accuracy: 0.9786 - val_loss: 0.0711 - val_accuracy: 0.9805


In [5]:
import tensorflow_model_optimization as tfmot

prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude

# Compute end step to finish pruning after 2 epochs.
batch_size = 128
epochs = 2
validation_split = 0.1 # 10% of training set will be used for validation set.

num_images = train_images.shape[0] * (1 - validation_split)
end_step = np.ceil(num_images / batch_size).astype(np.int32) * epochs

# Define model for pruning.
pruning_params = {
      'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.50,
                                                               final_sparsity=0.80,
                                                               begin_step=0,
                                                               end_step=end_step)
}

model_for_pruning = prune_low_magnitude(model, **pruning_params)

# `prune_low_magnitude` requires a recompile.
model_for_pruning.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model_for_pruning.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 prune_low_magnitude_reshape  (None, 28, 28, 1)        1         
  (PruneLowMagnitude)                                            
                                                                 
 prune_low_magnitude_conv2d   (None, 26, 26, 12)       230       
 (PruneLowMagnitude)                                             
                                                                 
 prune_low_magnitude_max_poo  (None, 13, 13, 12)       1         
 ling2d (PruneLowMagnitude)                                      
                                                                 
 prune_low_magnitude_flatten  (None, 2028)             1         
  (PruneLowMagnitude)                                            
                                                                 
 prune_low_magnitude_dense (  (None, 10)               4

# pruning은 decision tree에서 연산을 오래걸리게 하는 요소를 없애는 방법으로 쓰인 적이 있다.

# Mixed precision : float32(기본)과 float16을 섞어 쓸 수 있다. GPU가 좋아야 가능

# quantization : 양자화
학습은 float32로 시키고 모델은 int까지 범위를 줄여나가는 기법
그리고 재학습을 꼭 시킨다. 그래야 그 숫자 범위에 대해서는 더 정확한 결과가 나옴
IoT기기는 float가 연산을 못하는 경우가 많다.

In [6]:
# float32가 아니면 미분이 안됨 -> 모델이 만들어지지 않음
import tensorflow as tf

In [11]:
x = tf.Variable([1,2,3,4], dtype=tf.float32)
with tf.GradientTape() as t:
  y = x*x

In [12]:
t.gradient(y,x)

<tf.Tensor: shape=(4,), dtype=float32, numpy=array([2., 4., 6., 8.], dtype=float32)>

# Neural Architecture Search with reinforcement learning(NAS) : 강화학습을 활성화시킨 기법 - metamodelling
모델 구조를 최적화하는 모델